# FCIDECOMP software usage examples

This notebooks presents examples of the different usage patterns of the FCIDECOMP software.

## Introduction

The FCIDECOMP software enable users to read NetCDF files compressed with the JPEG-LS compression filter. Once installed, it can be used with:

- Python libraries, such as ``xarray`` adn ``netcdf4``
- netCDF-C based CLI tools, such as ``nccopy``
- The EUMETSAT Data Tailor, through the ``epct_plugin_mtg`` plugin
- Some netCDF-Java based applications, such as Panoply

In the following sections, usage examples of the first three points above are reported. For usage with NetCDF-Java based applications, see the README file.

In order to run the following examples, ensure that the ``HDF5_PLUGIN_PATH`` is set to the path of the directory containing the FCIDECOMP software libraries, and that the ``EPCT_TEST_DATA_DIR`` is set as described in the ``README`` accompanying this notebook.

## General imports and setting

In the following, general purpose Python libraries and the ``fcidecomp`` Python module are loaded

**NOTE**: if the FCIDECOMP software has been correctly installed and the ``HDF5_PLUGIN_PATH`` has been set to the right location, no error should be raised

In [ ]:
import os

The following line is needed to show plots in the notebook

In [ ]:
%matplotlib inline

Let's assign to the `TEST_FILE_DIR` variable the path to directory containing the test JPEG-LS compressed NetCDF file

In [ ]:
TEST_FILE_DIR = os.path.join(os.environ["EPCT_TEST_DATA_DIR"], "MTG", "MTGFCIL1")
COMP_FILE = os.path.join(
    TEST_FILE_DIR,
    "W_XX-EUMETSAT-Darmstadt_IMG+SAT_MTI1+FCI-1C-RRAD-FDHSI-FD--CHK-BODY--"
    "DIS-NC4E_C_EUMT_20200405120015_GTT_DEV_20200405115500_20200405115515_N_JLS_T_0072_0021.nc"
)

## Usage with Python

### ``xarray``

Let's import both the ``xarray`` and ``fcidecomp`` libraries, which loads the FCIDECOMP decompression filter.

In [ ]:
import xarray as xr

import fcidecomp

Let's open an JPEG-LS compressed file, loading a specific group of the netCDF file as an ``xarray`` dataset 

In [ ]:
ds = xr.open_dataset(COMP_FILE, group='data/ir_105/measured')
ds

Now let's plot the ``effective_radiace`` variable, showing that the file has been correctly uncompressed.

In [ ]:
ds.effective_radiance.plot()

### ``netCDF4``

Let's import both the ``matplotlib``, ``netCDF4`` and ``fcidecomp`` libraries, which loads the FCIDECOMP decompression filter.

In [ ]:
import matplotlib.pyplot as plt
import netCDF4 as nc

import fcidecomp

Now open the JPEG-LS compressed netCDF file and load it into the ``ds`` variable

In [ ]:
ds = nc.Dataset(COMP_FILE, "r")

Let's extract values of the band ``'data/ir_105/measured/effective_radiance`` and assign the resulting masked array to the ``band`` variable, and let's print its content

In [ ]:
band = ds['data/ir_105/measured/effective_radiance'][:]
band

## Usage with netCDF-C based CLI tools

### ``nccopy``

The following line is actually run in the shell

In [ ]:
!nccopy -F none {COMP_FILE} {os.path.join(TEST_FILE_DIR, "uncompressed_nccopy.nc")}

Let's open the output of the previous shell command, which create a decompressed version of the JPEG-LS compressed test file ``COMP_FILE``

In [ ]:
ds = xr.open_dataset(os.path.join(TEST_FILE_DIR, "uncompressed_nccopy.nc"), group='data/ir_105/measured')
ds

Now let's plot the ``effective_radiace`` variable, showing that the file has been correctly uncompressed

In [ ]:
ds.effective_radiance.plot()

## Usage with the EUMETSAT Data Tailor

Decompression of JPEG-LS compressed netCDF files with the EUMETSAT Data Tailor software is enabled through its ``epct_plugin_mtg`` plugin.

First, let's import the ``epct.api`` module and the ``xarray`` library

In [ ]:
import xarray as xr

from epct import api

Let's check that the plugin is correctly installed

In [ ]:
api.info()

Let's run the chain to decompress the JPEG-LS compressed netCDF file ``COMP_FILE``

In [ ]:
chain_config = {
    "product":   "MTGFCIL1", 
    "format": "netcdf4_satellite"
}
target_dir = './'
product_paths = [COMP_FILE]
output_products = api.run_chain(
    product_paths, 
    chain_config=chain_config, 
    target_dir=target_dir, 
)

Let's open the decompressed output's group ``data/ir_105/measured`` as an ``xarray`` dataset and plot its ``effective_radiance`` variable

In [ ]:
ds = xr.open_dataset(output_products[0], group='data/ir_105/measured')
ds.effective_radiance.plot()